In [7]:
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer
from energy_bert.data import BertDataBunch
from energy_bert.learner import BertLearner
from energy_bert.utils.spellcheck import BingSpellCheck
from pathlib import Path

In [8]:
# !pip --no-cache-dir install git+https://e791691795db788356f2d576c50aa90829425c7e@github.com/kaushaltrivedi/energy-bert.git --upgrade

In [2]:
MODEL_PATH=Path('../models/intent_classification_lib_2019-02-17_23-26-25.bin')
BERT_PRETRAINED_PATH = Path('../../bert_models/pretrained-weights/uncased_L-12_H-768_A-12/')
LABEL_PATH = Path('../labels')

In [3]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=True)

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
device

device(type='cuda')

In [6]:
databunch = BertDataBunch(LABEL_PATH, LABEL_PATH, tokenizer, train_file=None, val_file=None,
                          bs=32, maxlen=512, multi_gpu=False, multi_label=False)

In [7]:
num_labels = len(databunch.labels)

In [8]:
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_PATH, [], device, None, 
                                            finetuned_wgts_path=MODEL_PATH, 
                                            is_fp16=False, loss_scale=128, multi_label=False)

In [9]:
spellcheck = BingSpellCheck('8cf9679697c9464c881be6c2350f4bbd')

In [37]:
# %%timeit
text = """
please log a compliant
"""
text = spellcheck.spell_check(text)
learner.predict_batch([text])

[[('enquire-complaint', 0.9972952),
  ('problem-vague', 0.0001993357),
  ('dispute-bill', 0.00014630113),
  ('enquire-smart_meter', 0.00013876527),
  ('enquire-bill_final', 0.0001315836),
  ('vague-plumbing', 9.7592696e-05),
  ('report-appointment_missed', 6.630923e-05),
  ('report-letter', 6.0486927e-05),
  ('enquire-bill_energy', 5.795332e-05),
  ('enquire-vague', 4.5781017e-05),
  ('pay-bill_final', 4.1911975e-05),
  ('enquire-hive', 4.0076764e-05),
  ('report-meter_reading_final', 3.9258e-05),
  ('enquire-debt', 3.8071506e-05),
  ('enquire-service', 3.63215e-05),
  ('report-bill_unpaid', 3.5249424e-05),
  ('vague-breakdown', 3.514527e-05),
  ('enquire-appointment_today', 3.4814613e-05),
  ('problem-toilet', 3.4333912e-05),
  ('report-home_move', 3.0948166e-05),
  ('report-property_new', 2.9598303e-05),
  ('report-leak_water', 2.9557175e-05),
  ('request-appointment', 2.8302216e-05),
  ('enquire-appointment', 2.8284407e-05),
  ('request-advisor', 2.7621007e-05),
  ('enquire-appointm

In [29]:
# text = spellcheck.spell_check('the worl is not enuf')
text

'\ncan i get my supplier\n'